<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/LineModell_japanese_large_lm_3_6b_instruction_sf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/line-corporation/https://huggingface.co/line-corporation/japanese-large-lm-1.7b-instruction-sft

In [1]:
# パッケージのインストール
!pip install -q transformers accelerate bitsandbytes
!pip install -q sentencepiece einops

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "line-corporation/japanese-large-lm-3.6b-instruction-sft",
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    "line-corporation/japanese-large-lm-3.6b-instruction-sft",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [12]:
def generate_text(input_text):

  input_ids = tokenizer.encode(
      f"ユーザー: {input_text}\nシステム: ",
      add_special_tokens=False,
      return_tensors="pt",
      )

  tokens = model.generate(
      input_ids.to(device=model.device),
      max_length=1028,
      temperature=0.7,
      do_sample=True,
      top_p = 0.9,
      top_k = 0,
      repetition_penalty = 1.1,
      num_beams = 1,
      pad_token_id=tokenizer.pad_token_id,
      )

  return tokenizer.decode(tokens[0])

In [13]:
%%time
print(generate_text("今日の天気と夕飯の関係を考察してください。"))

ユーザー: 今日の天気と夕飯の関係を考察してください。
システム: 1.曇り時々晴れ、気温26度、湿度70%。
     ・湿度が高く蒸し暑いため、ビールや冷酒が飲みたくなるかもしれません!
      ・しかし、我慢です!今夜は鍋にしましょう!

2.雨時々曇り、気温23度、湿度59%。
     ・湿度が低く涼しいので、鍋よりも冷たい飲み物の方が合いそうです。
     ・しかし、我慢です!今夜は白ワインにしましょう。

3.曇り時々晴れ、気温26度、湿度58%。
     ・湿度は低いですが、気温が高いので、温かい食事が合うでしょう。
     ・今夜は、ガーリックの効いたスペイン風のパエリアを用意しましょう!</s>
CPU times: user 26.2 s, sys: 0 ns, total: 26.2 s
Wall time: 26.5 s


In [14]:
%%time
print(generate_text("まどか☆マギカで一番かわいいのは誰？"))

ユーザー: まどか☆マギカで一番かわいいのは誰?
システム: ほむらです。</s>
CPU times: user 869 ms, sys: 0 ns, total: 869 ms
Wall time: 867 ms


In [15]:
%%time
print(generate_text("卿らの討議も長いが、結論は出たのか？"))

ユーザー: 卿らの討議も長いが、結論は出たのか?
システム: 卿らの討議も長いが、結論が出たのか?
我々は、善と悪という概念そのものを問い直さなければならない。
善悪二元論にしがみつくことで、私たちは多くのものを失ってきたのです。</s>
CPU times: user 7.81 s, sys: 0 ns, total: 7.81 s
Wall time: 7.8 s


In [16]:
%%time
print(generate_text("九州でもっとも人口が多い県は？"))

ユーザー: 九州でもっとも人口が多い県は?
システム: 2021年国勢調査によると、九州で最も人口の多い県は熊本県です。その人口は約96万人である。</s>
CPU times: user 5.46 s, sys: 154 µs, total: 5.46 s
Wall time: 5.81 s


In [17]:
%%time
print(generate_text(
"""
「美味しいご飯たくさん食べれて、嬉しいす」を正しい日本語に校正してください。
"""))

ユーザー: 
「美味しいご飯たくさん食べれて、嬉しいす」を正しい日本語に校正してください。

システム: 美味しくご飯を食べられてうれしいです!</s>
CPU times: user 1.03 s, sys: 3.72 ms, total: 1.04 s
Wall time: 1.04 s
